In [22]:
font_list = [ 
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans-Bold.ttf',
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSansMono-Bold.ttf',
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSansMono.ttf',
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf',
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSerif-Italic.ttf',
    '/usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSerif.ttf',
 ]
 
import random
import PIL
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

font = ImageFont.truetype(random.choice(font_list), size=random.randint(100, 200))
font

In [23]:
import os

if not os.path.exists("./data/test"): os.mkdir("./data/test")
if not os.path.exists("./data/train"): os.mkdir("./data/train")

Пилим пикчу на составные и работаем с ними как с отдельными изображениями

In [24]:
# PIL.Image.MAX_IMAGE_PIXELS = 900000000

# img = Image.open('/home/owner/Documents/DEV/DL/hw-letters-xCthulhuFtagn/letters_58.png')

# for i in range(100):
#     for j in range(100):
#         new_img = img.crop((300*i,300*j, 300*(i+1), 300*(j+1)))
#         new_img.save(f"./data/test/letter_{(i+1) + j*100}.jpeg")

Генерим свои данные

In [25]:
# # https://stackoverflow.com/questions/75344148/trying-to-use-python-pil-imagedraw-textbbox-because-of-textsize-deprecation-but
# from string import ascii_uppercase
# from random import randint
# import math
# import numpy as np

# classes = (set(ascii_uppercase) - {'N', 'Z'} - {'M', 'W'}).union({"N_Z", "M_W"})
# for letter in classes:
#     for i in range(100):
#         empty = Image.new('RGBA', (300, 300), (255, 255,255, 255))
#         overlay = Image.new('RGBA', (300, 300), (255, 255, 255, 0))
        
#         empty_draw = ImageDraw.Draw(empty)
#         n_circles = randint(10, 40)
#         for _ in range(n_circles):
#             circle = Image.new('RGBA', (300, 300), (255, 255,255, 0))
#             circle_draw = ImageDraw.Draw(circle)
#             color = (random.randint(0, 200), 
#                      random.randint(0, 200), 
#                      random.randint(0, 200),
#                      random.randint(50, 70))
#             x = randint(10, 290)
#             y = randint(10, 290)
#             radius = randint(1, 35)
#             xy = [x - radius, y - radius, x + radius, y + radius]
#             circle_draw.ellipse(xy, fill=color)
#             empty = Image.alpha_composite(empty, circle)
        
#         overlay_draw = ImageDraw.Draw(overlay)
#         spin_angle = randint(0, 359)
#         radians = math.radians(spin_angle)
#         spread = 0
#         x, y = 150 + randint(-spread, spread), 150 + randint(-spread, spread)
        
#         if letter == "N_Z": label = "N" if np.random.random() < 0.5 else "Z"
#         elif letter == "M_W": label = "M" if np.random.random() < 0.5 else "W"
#         else: label = letter
        
#         overlay_draw.text(
#             (x, y),
#             text=label,
#             font=font,
#             fill=(randint(0, 255), randint(0, 255), randint(0, 255), 255),
#         )
        
#         new_img = Image.alpha_composite(empty, overlay.rotate(spin_angle))
#         new_img = new_img.convert('RGB')
#         new_img.save(f"./data/train/{letter}_{i}.png", 'PNG')


Херачим модель

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import os
import numpy as np
from string import ascii_uppercase

class DatasetLetters(Dataset):
    def __init__(self, root='./data', train=True, transform=None):
        self.root = root
        self.transform = transform
        self.classes = list(set(ascii_uppercase) - {'N', 'Z'} - {'M', 'W'})
        self.classes.sort()
        self.classes.insert(12, "N_Z")
        self.classes.insert(12, "M_W")
        self.classes = dict(zip(range(len(self.classes)), self.classes))
        self.len = len(os.listdir(self.root + "/train"))
        self.n_examples = 100

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        # load image
        # transform image
        # return dict with image and class label
        letter = self.classes[index // self.n_examples]
        sub_index = index % self.n_examples
        file_name = f"{letter}_{sub_index}.png"

        pic = cv2.imread(self.root + "/train/" + file_name)
        # print(f"label:{index // self.n_examples}")
        return {"image": self.transform(pic), "label": torch.Tensor([index // self.n_examples]).type(torch.LongTensor)}

In [27]:
import torch.nn as nn
import torch.nn.functional as F

class LetterCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=3) #100x100x6
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=1, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=4) #25x25x4
        self.lin = nn.Linear(in_features=625, out_features=100)
        self.final = nn.Linear(in_features=100, out_features=24)
    
    def forward(self, x : torch.Tensor):
        # print(f"x: {x.shape}")
        x.to("cuda:0")
        x = F.leaky_relu(self.conv1(x))
        x = self.pool1(x)
        x = F.leaky_relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.lin(x.flatten(2)))
        return self.final(x)

In [28]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class SmallerLetterCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=1, kernel_size=3, padding=1)
        self.final = nn.Linear(in_features=300*300, out_features=24)
    
    def forward(self, x : torch.Tensor):
        # print(f"x: {x.shape}")
        x.to("cuda:0")
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return self.final(x.flatten(2))

In [29]:
from torchsummary import summary

summary(model=SmallerLetterCNN().to("cuda:0"), input_size=(3,300,300), batch_size=128)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [128, 4, 300, 300]             112
            Conv2d-2         [128, 1, 300, 300]              37
            Linear-3               [128, 1, 24]       2,160,024
Total params: 2,160,173
Trainable params: 2,160,173
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 131.84
Forward/backward pass size (MB): 439.48
Params size (MB): 8.24
Estimated Total Size (MB): 579.55
----------------------------------------------------------------


In [30]:
summary(model=LetterCNN().to("cuda:0"), input_size=(3,300,300), batch_size=128)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [128, 6, 300, 300]             168
         MaxPool2d-2         [128, 6, 100, 100]               0
            Conv2d-3         [128, 1, 100, 100]              55
         MaxPool2d-4           [128, 1, 25, 25]               0
            Linear-5              [128, 1, 100]          62,600
            Linear-6               [128, 1, 24]           2,424
Total params: 65,247
Trainable params: 65,247
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 131.84
Forward/backward pass size (MB): 596.43
Params size (MB): 0.25
Estimated Total Size (MB): 728.52
----------------------------------------------------------------


In [31]:
# from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
import os
import pandas as pd

class Trainer:
    def __init__(self, model, optimizer, train_dataset, batch_size=128):
        self.model = model
        self.optimizer = optimizer
        self.train_dataset = train_dataset
        self.lossF = nn.CrossEntropyLoss(reduction='none')

        self.batch_size = batch_size

        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = self.model.to(self.device)

        self.global_step = 0

    def save_checkpoint(self, path):
        torch.save(self.model.state_dict(), path)
        
    def load_checkpoint(self, path):
        self.model.load_state_dict(torch.load(path))

    def train(self, num_epochs):
        model = self.model
        optimizer = self.optimizer

        train_loader = DataLoader(self.train_dataset, shuffle=True, pin_memory=True, batch_size=self.batch_size, num_workers=12)

        for epoch in tqdm(range(num_epochs)):
            model.train()
            for batch in train_loader:
                x = batch["image"].to(self.device)
                y = torch.Tensor(batch["label"]).to(self.device).squeeze()
                
                logits = model.forward(x).squeeze()
                # print(f"logits: shape = {logits.shape}")
                # print(logits)
                # print(f"target: shape = {y.shape}")
                # print(y)
                
                loss = self.lossF(logits, y)

                optimizer.zero_grad()
                loss.sum().backward()
                optimizer.step()

                self.global_step += 1
                
    def classify(self):
        path = "/home/owner/Documents/DEV/DL/hw-letters-xCthulhuFtagn/data/test"
        self.model.eval()
        classes = self.train_dataset.classes
        
        letters_df = pd.read_csv("/home/owner/Documents/DEV/DL/hw-letters-xCthulhuFtagn/letters.csv", header=None)
        letters_df[1] = 0
        display(letters_df)
        
        for filename in os.listdir(path):
            pic = torch.Tensor(cv2.imread(path + "/" + filename))
            pic = pic.movedim(-1, 0).unsqueeze(0)
            pic = pic.to(self.device)
            logits = self.model(pic)
            letter = classes[logits.argmax().item()]
            if letter == "N_Z":
                if np.random.random() < 0.5: letters_df.loc[letters_df[0] == "N", 1] += 1
                else: letters_df.loc[letters_df[0] == "Z", 1] += 1
            elif letter == "M_W":
                if np.random.random() < 0.5: letters_df.loc[letters_df[0] == "M", 1] += 1
                else: letters_df.loc[letters_df[0] == "W", 1] += 1
            else: letters_df.loc[letters_df[0] == letter, 1] += 1
        
        letters_df.to_csv("/home/owner/Documents/DEV/DL/hw-letters-xCthulhuFtagn/letters.csv", index=False, header=None)

    def try_pic(self, path):
        pic = torch.Tensor(cv2.imread(path))
        pic = pic.movedim(-1, 0).unsqueeze(0)
        pic = pic.to(self.device)
        logits = self.model(pic)
        letter = self.train_dataset.classes[logits.argmax().item()]
        print(self.train_dataset.classes)
        print(f"index {logits.argmax().item()}; class {letter}")

In [32]:
from torchvision import transforms

transform_to_tensor = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = DatasetLetters(transform=transform_to_tensor)
model = SmallerLetterCNN()

trainer = Trainer(model, torch.optim.Adam(model.parameters(), lr=0.7), train_dataset, batch_size=64)

In [33]:
torch.cuda.is_available()

True

In [34]:
trainer.train(10000)

  0%|          | 5/10000 [00:18<10:28:57,  3.78s/it]


KeyboardInterrupt: 

In [ ]:
trainer.save_checkpoint("model.pkl")
# trainer.load_checkpoint("model.pkl")

In [ ]:
trainer.try_pic("/home/owner/Documents/DEV/DL/hw-letters-xCthulhuFtagn/data/test/letter_9994.jpeg")

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M_W', 13: 'N_Z', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'X', 23: 'Y'}
index 6; class G


In [ ]:
trainer.classify()

,0,1
0,A,0
1,B,0
2,C,0
3,D,0
4,E,0
5,F,0
6,G,0
7,H,0
8,I,0
9,J,0


In [ ]:
pd.read_csv("/home/owner/Documents/DEV/DL/hw-letters-xCthulhuFtagn/letters.csv", header=None)

,0,1
0,A,0
1,B,0
2,C,0
3,D,0
4,E,0
5,F,0
6,G,10000
7,H,0
8,I,0
9,J,0
